## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd
df = pd.read_csv('train/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df.drop('id', axis='columns', inplace=True)
df.head()

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.isna().sum()

title      558
author    1957
text        39
label        0
dtype: int64

In [4]:
# Drop Nan Values
df = df.dropna()

In [5]:
df.isna().sum()

title     0
author    0
text      0
label     0
dtype: int64

In [6]:
# Get the Independent Features
X = df.drop('label',axis=1)

In [7]:
## Get the Dependent features
y = df['label']

In [8]:
X.shape

(18285, 3)

In [9]:
y.shape

(18285,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.4.1'

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
# Vocabulary size
voc_size=5000

### Onehot Representation

In [14]:
messages = X.copy()

In [15]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [16]:
# As we dropped the na values we should reset the index 
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
# Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, 1):
    print(messages['title'][i])
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    print(review)
    review = review.lower()
    print(review)
    review = review.split()
    print(review)
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    print(review)
    review = ' '.join(review)
    print(review)
    corpus.append(review)

House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It
House Dem Aide  We Didn t Even See Comey s Letter Until Jason Chaffetz Tweeted It
house dem aide  we didn t even see comey s letter until jason chaffetz tweeted it
['house', 'dem', 'aide', 'we', 'didn', 't', 'even', 'see', 'comey', 's', 'letter', 'until', 'jason', 'chaffetz', 'tweeted', 'it']
['hous', 'dem', 'aid', 'even', 'see', 'comey', 'letter', 'jason', 'chaffetz', 'tweet']
hous dem aid even see comey letter jason chaffetz tweet


In [19]:
# Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus[0:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [21]:
onehot_repr = [one_hot(words, voc_size) for words in corpus] 
onehot_repr[0:5]

[[4657, 623, 4745, 3327, 2233, 2905, 1440, 2953, 2057, 4154],
 [4848, 4011, 402, 4239, 3801, 2517, 3605],
 [4598, 936, 83, 4867],
 [685, 543, 2062, 3558, 721, 2274],
 [1522, 3801, 708, 4391, 3838, 3462, 3801, 1440, 1081, 4023]]

### Embedding Representation

In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr, padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 2953, 2057, 4154],
       [   0,    0,    0, ..., 3801, 2517, 3605],
       [   0,    0,    0, ...,  936,   83, 4867],
       ...,
       [   0,    0,    0, ..., 3621, 3874,  790],
       [   0,    0,    0, ..., 3381, 4174,   24],
       [   0,    0,    0, ..., 3842,  939, 2103]])

In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4657,
        623, 4745, 3327, 2233, 2905, 1440, 2953, 2057, 4154])

In [24]:
# Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=20))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [25]:
# since the final output is sigmoid we are getting 1output
# predicting without training with random weight intialization
model.predict(embedded_docs).shape

(18285, 1)

In [26]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [27]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [29]:
# Finally Training
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 20s 67ms/step - loss: 0.4917 - accuracy: 0.7357 - val_loss: 0.1949 - val_accuracy: 0.9158
Epoch 2/10
192/192 [==============================] - 8s 41ms/step - loss: 0.1306 - accuracy: 0.9512 - val_loss: 0.2408 - val_accuracy: 0.9168
Epoch 3/10
192/192 [==============================] - 8s 41ms/step - loss: 0.0954 - accuracy: 0.9608 - val_loss: 0.2295 - val_accuracy: 0.9160
Epoch 4/10
192/192 [==============================] - 8s 41ms/step - loss: 0.0658 - accuracy: 0.9778 - val_loss: 0.2698 - val_accuracy: 0.9185
Epoch 5/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0526 - accuracy: 0.9837 - val_loss: 0.2694 - val_accuracy: 0.9100
Epoch 6/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0405 - accuracy: 0.9871 - val_loss: 0.3404 - val_accuracy: 0.9127
Epoch 7/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0288 - accuracy: 0.9917 - val_loss: 0.3465 - val_accuracy: 0.906

### Performance Metrics And Accuracy

In [30]:
y_pred = model.predict(X_test)
y_pred[0:5]

array([[9.9998486e-01],
       [2.9747559e-06],
       [1.3411715e-02],
       [1.4344512e-02],
       [9.9996197e-01]], dtype=float32)

In [31]:
y_pred = np.round(y_pred)
y_pred[0:5]

array([[1.],
       [0.],
       [0.],
       [0.],
       [1.]], dtype=float32)

In [32]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3097,  322],
       [ 227, 2389]], dtype=int64)

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9090306545153273

### Adding Dropout 

In [34]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
modeld=Sequential()
modeld.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
modeld.add(Dropout(0.4))
modeld.add(LSTM(100))
modeld.add(Dropout(0.5))
modeld.add(Dense(1,activation='sigmoid'))
modeld.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Finally Training
modeld.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modeld.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 15s 47ms/step - loss: 0.5002 - accuracy: 0.7383 - val_loss: 0.2260 - val_accuracy: 0.9032
Epoch 2/10
192/192 [==============================] - 8s 40ms/step - loss: 0.1601 - accuracy: 0.9379 - val_loss: 0.1937 - val_accuracy: 0.9185
Epoch 3/10
192/192 [==============================] - 6s 30ms/step - loss: 0.1142 - accuracy: 0.9582 - val_loss: 0.2120 - val_accuracy: 0.9191
Epoch 4/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0914 - accuracy: 0.9665 - val_loss: 0.2283 - val_accuracy: 0.9150
Epoch 5/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0715 - accuracy: 0.9748 - val_loss: 0.2693 - val_accuracy: 0.9047
Epoch 6/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0592 - accuracy: 0.9800 - val_loss: 0.2768 - val_accuracy: 0.9112
Epoch 7/10
192/192 [==============================] - 8s 44ms/step - loss: 0.0494 - accuracy: 0.9838 - val_loss: 0.2955 - val_accuracy: 0.911

In [36]:
y_pred = model.predict(X_test)
y_pred[0:5]

array([[9.9998486e-01],
       [2.9747559e-06],
       [1.3411715e-02],
       [1.4344512e-02],
       [9.9996197e-01]], dtype=float32)

In [37]:
y_pred = np.round(y_pred)
y_pred[0:5]

array([[1.],
       [0.],
       [0.],
       [0.],
       [1.]], dtype=float32)

In [38]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3097,  322],
       [ 227, 2389]], dtype=int64)

In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9090306545153273